# Building Custom Tool using Claude 3.5 Sonnet 

### Install required libraries

In [4]:
pip install -q anthropic


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Upgrading existing libraries

In [7]:
pip install --upgrade -q anthropic


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: /opt/anaconda3/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Show current version

In [10]:
pip show anthropic

Name: anthropic
Version: 0.31.2
Summary: The official Python library for the anthropic API
Home-page: 
Author: 
Author-email: Anthropic <support@anthropic.com>
License: 
Location: /opt/anaconda3/lib/python3.11/site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tokenizers, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Load Environment Variable

In [13]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

### Simple Message Request

In [24]:
from anthropic import Anthropic
client = Anthropic()

model_name = 'claude-3-5-sonnet-20240620'

response = client.messages.create(
    model=model_name,
    max_tokens=1024,
    messages=[
        {
            'role':'user',
            #'content':'Hello world!'
            'content':'Write a short story about the life in the moon'
        }
    ],
    system='You are a creative assistant that only respond with english language',
    stop_sequences=["The end."],
    temperature=0.9
)

print(response.content[0].text)

Luna's Lament

Aria gazed out the reinforced window of Lunar Base Alpha, her eyes tracing the stark contrast between the inky blackness of space and the pale, pockmarked surface of the Moon. She had been stationed here for three years now, part of the first long-term human settlement beyond Earth.

Life on the Moon was a constant challenge. The harsh environment demanded unwavering vigilance. Every breath of recycled air, every sip of purified water, and every morsel of hydroponic food was a testament to human ingenuity and perseverance.

Aria's daily routine was a delicate dance of maintaining life support systems, conducting scientific experiments, and battling the psychological toll of isolation. The Earth, a beautiful blue marble suspended in the void, was a constant reminder of what they had left behind.

Despite the hardships, there were moments of breathtaking beauty. Earthrise never failed to leave Aria in awe, and the long lunar nights provided unparalleled views of the cosmos

# Calculator Tool

### 1. Define Calculator Implementation

In [36]:
def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

### Test the function

In [39]:
calculator("add", 5 , 10)

15

### 2. Calculator Tool Definition

In [43]:
calculator_tool = {
    "name":"calculator",
    "description":"A simple calculator that performs basic arithmetic operations",
    "input_schema":{
        "type":"object",
        "properties":{
            "operation":{
                "type":"string",
                "enum":["add","subtract","multiply","divide"],
                "description":"The arithemtic operation to perform"
            },
            "operand1":{
                "type":"number",
                "description":"The first operand."
            },
            "operand2":{
                "type":"number",
                "description":"The second operand"
            }
        },
        "required":["operation","operand1","operand2"]
    }
}

### 3. Providing Claude with our Tool

In [45]:
from anthropic import Anthropic
client = Anthropic()

model_name = 'claude-3-5-sonnet-20240620'

response = client.messages.create(
    model=model_name,
    messages=[
        {
            'role':'user',
            'content':'Multiply 1984135 by 934316. Only respond with the result'
        }
    ],
    max_tokens=1024,
    tools=[calculator_tool]
)

print(response)

Message(id='msg_01LMzt2LvwjxZFUHDwSxu8sY', content=[TextBlock(text="Certainly! To multiply these two large numbers, I'll use the calculator function. Let me perform that calculation for you.", type='text'), ToolUseBlock(id='toolu_0187zNGeFxK4FnXC7GQMZRjG', input={'operation': 'multiply', 'operand1': 1984135, 'operand2': 934316}, name='calculator', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=452, output_tokens=117))


In [49]:
response.stop_reason

'tool_use'

In [51]:
response.content

[TextBlock(text="Certainly! To multiply these two large numbers, I'll use the calculator function. Let me perform that calculation for you.", type='text'),
 ToolUseBlock(id='toolu_0187zNGeFxK4FnXC7GQMZRjG', input={'operation': 'multiply', 'operand1': 1984135, 'operand2': 934316}, name='calculator', type='tool_use')]

In [57]:
tool_name = response.content[1].name
tool_inputs = response.content[1].input

print("The tool name claude wants to call is:", tool_name)
print("The inputs claude want to call it with are:", tool_inputs)

The tool name claude wants to call is: calculator
The inputs claude want to call it with are: {'operation': 'multiply', 'operand1': 1984135, 'operand2': 934316}


### Putting it all together

In [59]:
def calculator(operation, operand1, operand2):
    if operation == "add":
        return operand1 + operand2
    elif operation == "subtract":
        return operand1 - operand2
    elif operation == "multiply":
        return operand1 * operand2
    elif operation == "divide":
        if operand2 == 0:
            raise ValueError("Cannot divide by zero.")
        return operand1 / operand2
    else:
        raise ValueError(f"Unsupported operation: {operation}")

calculator_tool = {
    "name":"calculator",
    "description":"A simple calculator that performs basic arithmetic operations",
    "input_schema":{
        "type":"object",
        "properties":{
            "operation":{
                "type":"string",
                "enum":["add","subtract","multiply","divide"],
                "description":"The arithemtic operation to perform"
            },
            "operand1":{
                "type":"number",
                "description":"The first operand."
            },
            "operand2":{
                "type":"number",
                "description":"The second operand"
            }
        },
        "required":["operation","operand1","operand2"]
    }
}


def prompt_claude(prompt):
    response = client.messages.create(
        model=model_name,
        messages = [
            {
                'role':'user',
                'content':prompt
            }
        ],
        system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal.",
        max_tokens=500,
        tools=[calculator_tool]
    )

    if response.stop_reason == "tool_use":
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input

        if tool_name == "calculator":
            print("Claude wants to use the calculator tool")
            operation = tool_input["operation"]
            operand1 = tool_input["operand1"]
            operand2 = tool_input["operand2"]

            try:
                result = calculator(operation, operand1, operand2)
                print("Calculation result is:", result)
            except ValueError as e:
                print(f"Error: {str(e)}")
    else:
        print("Claude didn't want to use a tool")
        print("Claude responded with:")
        print(response.content[0].text)

In [63]:
prompt_claude("What is 348.99 times 34")

Claude wants to use the calculator tool
Calculation result is: 11865.66


In [65]:
prompt_claude("Write a hiaku about the moon")

Claude didn't want to use a tool
Claude responded with:
I apologize, but I don't have a specific tool for writing haiku or poetry. However, I'd be happy to compose a haiku about the moon for you using my general language capabilities. Here's a haiku about the moon:

Silvery moonlight
Bathes the quiet night in dreams
Celestial guide

This haiku follows the traditional 5-7-5 syllable pattern and captures the serene, dreamlike quality often associated with the moon. The moon is portrayed as a celestial guide, illuminating the night with its soft, silver light.


### Research Assistant Tool

In [ ]:
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("Files/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

In [ ]:
research_tool = {
    "name":"generate_wikipedia_reading_list",
    "description":"Generate a list of Wikipedia articles relevant to a research topic.",
    "input_schema":{
        "type":"object",
        "properties":{
            "research_topic":{
                "type":"string",
                "description":"The topic to research."
            },
            "article_titles":{
                "type":"array",
                "items":{"type":"string"},
                "description":"Titles of relevant Wikipedia articles.",
            },
        },
        "required":["research_topic","artilce_titles"],
    },
}

In [68]:
# Here's your starter code!
import wikipedia
def generate_wikipedia_reading_list(research_topic, article_titles):
    wikipedia_articles = []
    for t in article_titles:
        results = wikipedia.search(t)
        try:
            page = wikipedia.page(results[0])
            title = page.title
            url = page.url
            wikipedia_articles.append({"title": title, "url": url})
        except:
            continue
    add_to_research_reading_file(wikipedia_articles, research_topic)

def add_to_research_reading_file(articles, topic):
    with open("Files/research_reading.md", "a", encoding="utf-8") as file:
        file.write(f"## {topic} \n")
        for article in articles:
            title = article["title"]
            url = article["url"]
            file.write(f"* [{title}]({url}) \n")
        file.write(f"\n\n")

research_tool = {
    "name":"generate_wikipedia_reading_list",
    "description":"Generate a list of Wikipedia articles relevant to a research topic.",
    "input_schema":{
        "type":"object",
        "properties":{
            "research_topic":{
                "type":"string",
                "description":"The topic to research."
            },
            "article_titles":{
                "type":"array",
                "items":{"type":"string"},
                "description":"Titles of relevant Wikipedia articles.",
            },
        },
        "required":["research_topic","artilce_titles"],
    },
}
        
def get_research_help(topic, num_articles=3):
    prompt = (
       f"I need your help gathering research on the topic of '{topic}'."
       f"Please generate {num_articles} relevant Wikipedia article titles for me."
    )
    response = client.messages.create(
        model=model_name,
        system="You have access to tools, but only use them when necessary. If a tool is not required, respond as normal",
        messages=[{'role':'user', 'content':prompt}],
        max_tokens=500,
        tools=[research_tool],
    )
    if response.stop_reason == "tool_use":
        tool_use=response.content[-1]
        article_titles = tool_use.input.get("article_titles",[])
        if article_titles:
            generate_wikipedia_reading_list(topic, article_titles)
        else:
            print("Claude did not generate any article titles.")
    elif response.stop_reason == "end_turn":
        print("Claude responded with:")
        print(response.content[0].text)

In [70]:
get_research_help("Are animals conscious?")

In [72]:
get_research_help("Pirates Across the World")